In [340]:
from glob import glob
from pathlib import Path
import ROOT
import csv

In [341]:
absolute_path_only = '/media/droc/Seagate/PHYS-461/4-hall_effect/data/'
input_files = glob(absolute_path_only + '/*.root')

namer = []

for i in range(len(input_files)):

    base = Path(input_files[i]).stem
    namer.append(base[:base.index("_run")])

In [342]:
line = ROOT.TF1('line', '[0]*x+[1]', 0, 11)

pzero = []
pzero_error = []
pone = []
pone_error = []
curr = []
title = []

c0 = ROOT.TCanvas("c0","", 800,800,800,550)

def linear_fit(file_path,titlee):

    file = ROOT.TFile(file_path) 
    tree = file.Get("data")
    rdf = ROOT.RDataFrame(tree)

    bmeas = rdf.AsNumpy(columns=["Bmeas"])["Bmeas"][:-1]
    bnom = rdf.AsNumpy(columns=["Bnom"])["Bnom"][:-1]
    icoil = rdf.AsNumpy(columns=["Icoil"])["Icoil"]
    vhall = rdf.AsNumpy(columns=["VHall"])["VHall"][:-1]
    stddev = rdf.AsNumpy(columns=["stddev"])["stddev"]
    
    curr.append(float(stddev[-1]))
    stddev = stddev[:-1]

    # print(bmeas)
    # print(bnom)
    # print(stddev)
    # print(vhall)
    # print(icoil)
    file.Close()

    
    c0.Update()

    gr = ROOT.TGraphErrors(len(bmeas),bmeas,vhall,0,stddev)
    gr.SetMarkerStyle(ROOT.kFullCircle)
    gr.SetTitle(titlee)
    gr.GetXaxis().SetTitle("Measured Magnetic Field")
    gr.GetYaxis().SetTitle("Hall Voltage")
    gr.Fit(line, "Q")
    

    p0 = line.GetParameter(0)
    p0_error = line.GetParError(0)
    p1 = line.GetParameter(1)
    p1_error = line.GetParError(1)

    pzero.append(line.GetParameter(0))
    pzero_error.append(line.GetParError(0))
    pone.append(line.GetParameter(1))
    pone_error.append(line.GetParError(1))
    title.append(titlee)
    
    gr.Draw("AP")

    latex = ROOT.TLatex()
    latex.SetNDC()
    latex.SetTextSize(0.06)
    latex.DrawLatex(0.4,0.8,f'#scale[0.6]{{#bf{{slope: {p0:.2e} #sigma = #pm {p0_error:.2e} }}}}') 
    latex.DrawLatex(0.4,0.76,f'#scale[0.6]{{#bf{{y-intercept: {p1:.2e} #sigma = #pm {p1_error:.2e}}}}}') 

    png_file = file_path.replace(".root",".pdf")
    c0.SaveAs(png_file)
    c0.Clear()

    return pzero,pzero_error,pone,pone_error,curr,title

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c0


In [343]:
for index, file in enumerate(input_files):
    p0,p0_error,p1,p1_error,current,title = linear_fit(file,namer[index])

In [344]:
def lists_to_dict(keys, *values_lists):
    
    result_dict = {}
    thick = {"Mo": (0,0.0015), "Fe" : (0,0.0025), "Cu" : (0,0.006), "Ni": (0,0.001), "Cu-PCB": (0,0.0011), "Ge111": (0,0.035), "Ge100": (0,0.052), "Si": (0,0.053)}

    for i, key in enumerate(keys):
        values_tuple = tuple(values_list[i] for values_list in values_lists)
        result_dict[key] = values_tuple
        for j, elem in enumerate(thick):
            if elem in key:
                result_dict[key] = values_tuple + thick[elem]
    
    return result_dict

output_dict = lists_to_dict(namer, p0, p0_error,p1,p1_error,current)
# print(output_dict)

In [345]:
hall_resistance = {}

for index, i in enumerate(output_dict):
    
    slope = output_dict[i][0] * 1e-6
    y_intercept = output_dict[i][2] *1e-3

    hc = ( slope * output_dict[i][6] ) / output_dict[i][4]
    resistance = y_intercept / output_dict[i][4]

    hall_resistance[i] = (hc,resistance)

# print(hall_resistance)

In [346]:
all_data = '/media/droc/Seagate/PHYS-461/4-hall_effect/data/all_data.csv'

def dict_to_csv(input_dict, csv_filename):
    with open(csv_filename, mode='w', newline='') as file:
        writer = csv.writer(file)

        header = input_dict.keys()
        writer.writerow(header)

        rows = zip(*input_dict.values())
        writer.writerows(rows)

dict_to_csv(hall_resistance,all_data)